# **Notebook 2 - Generando texto con LSTMs.**


*   Recuerda que puedes consultar la documentación sobre una función escribiendo **?** justo después de la función: *Ejemplo: np.maximum?*
*   Puedes ejecutar el contenido de una celda con el atajo de teclado **CTRL+ENTER**
*   Utiliza **TAB** cada vez que quieras autocompletar una llamada a una función.
*   Puedes ejecutar instrucciones de bash directamente desde el notebook usando **!** : *Ejemplo: !pip install tensorflow*
*   Recuerda que Google es tu amigo, y saber buscar la información en las documentaciones de las librerías es muy importante.
*   Una solución correcta no es la que funciona sino la que se entiende!
*   No dudes en preguntar cualquier duda al profesor que lleva todo el día dando la turra.

## 1. Hoy vamos a generar '''_._._._._._._._._._.'''

- **¡Tokeniza tu texto!** - Utiliza la función *Tokenizer()* de Keras para convertir tu texto a vectores de etiqueta numérica. Observa el contenido del objeto ***tokenizer*** para ver qué información te aporta. De ahí, imprime los índices que se han generado para cada una de las palabras. ¿Qué ocurre cuando decodificamos una palabra que no aparece en nuestro vocabulario? ¿Son todas las palabras en nuestro diccionario cómo  esperamos que sean, o hace falta preprocesar y filtrar algunas palabras?  **Compruébalo!**

```
['Con', 'diez', 'cañones', ...] -----> [12, 1, 23, ...]
```

- **¡Crea tu dataset!** - A continuación toca generar tus variables de entrada y salida. Tus datos deberá de representar lo que esperamos predecir para el número de palabras dadas como entrada. Si de entrada sólo miramos 1 palabra para hacer la predicción (e.g. *X0 = ['Con'] ; Y0 = ['diez']*), se dice que estamos trabajando con 2-grama (un *bigrama*). Por ahora lo haremos así. Crea todos los posibles *bigramas*, y guarda en tu matriz **X** las palabras de entrada, y en la **Y** las palabras de salida. Recuerda que los índices de la salida deben de estar correctamente codificados (**One-Hot Encoded**) ¿Y los de entrada si usamos una capa de Embeddings?..

```
X : [12, 1, 23, ...] -----> Y: [1, 23, 4, ...]
```

  - **¡Crea tu modelo!** -  Vamos a crear un modelo sencillito que haga uso de lo visto hoy en clases. Tu modelo consistirá de una capa de *tf.keras.layers.Embeddings()* inicial. ¿Para qué sirve? A partir de ahí añade una capa *tf.keras.layers.LSTM()* y finalmente una capa densa con la capa de activación correspondiente.

- **¡Genera texto!** - Utiliza tu modelo para generar texto. Dale una palabra de tu vocabulario como comienzo y a partir de ahí utiliza cada texto de salida generado como input de la próxima iteración, así hasta obtener el texto de la longitud que quieras. ¿Qué te produce? ¿Observas algún fenómeno extraño? ¿Por qué?

- **¡Entiende lo que has hecho!** - Consulta en la documentación de Tensorflow las dimensiones de lo que estás utilizando; intenta mejorar el resultado del entrenamiento e investiga cómo podríamos mejorar el modelo. En las siguientes sesiones iremos añadiendo nuevos elementos al trabajo.




In [1]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

import numpy as np

In [2]:
!wget https://raw.githubusercontent.com/busiris2014/7506Condor1C2014/master/datos2011/trunk/libros/J.K.%20Rowling%20-%20Harry%20Potter%203%20-%20El%20Prisionero%20de%20Azkaban.txt

with open('/content/J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt', 'r') as file:
    data = file.read().replace('\n', ' ')

--2020-11-18 10:07:53--  https://raw.githubusercontent.com/busiris2014/7506Condor1C2014/master/datos2011/trunk/libros/J.K.%20Rowling%20-%20Harry%20Potter%203%20-%20El%20Prisionero%20de%20Azkaban.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 671651 (656K) [text/plain]
Saving to: ‘J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt’

J.K. Rowling - Harr 100%[===================>] 655.91K  --.-KB/s    in 0.02s   

2020-11-18 10:07:54 (26.4 MB/s) - ‘J.K. Rowling - Harry Potter 3 - El Prisionero de Azkaban.txt’ saved [671651/671651]



In [ ]:
!wget https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt

with open('/content/alice_in_wonderland.txt', 'r') as file:
    data = file.read().replace('\n', ' ')

--2020-11-17 12:35:16--  https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148574 (145K) [text/plain]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 145.09K  --.-KB/s    in 0.04s   

2020-11-17 12:35:17 (3.72 MB/s) - ‘alice_in_wonderland.txt’ saved [148574/148574]



In [4]:
Tokenizer?

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_vocab = 5000

tokenizer = Tokenizer(num_words=max_vocab)

# Añadimos al filtro el siguiente caracter, que se comprueba está presente en
# muchas palabras del vocabulario.
tokenizer.filters  = '"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n©\x92\x93\x94\x97«»'
# Configuramos al tokenizador.
tokenizer.fit_on_texts([data])
# Y codificamos nuestro texto
encoded = tokenizer.texts_to_sequences(data.split("."))

In [5]:
full_vocab_size = len(tokenizer.word_index) + 1

if not max_vocab:
  max_vocab = full_vocab_size

print('Tamaño del vocabulario:', full_vocab_size)

In [6]:
def decode_sequence(sequence):
  return " ".join([tokenizer.index_word.get(w) for w in sequence])

In [15]:
# Número de palabras en la frase a seleccionar.
n_tokens = 10

sequences = [] # Listado de secuencias a guardar.

# Para cada una de las frases del texto...
for sequence in encoded:
  # ...si la secuencia es mayor al número de tokens que queremos.
	if len(sequence) >= n_tokens:
			# ...nos desplazamos por la frase seleccionando subsecuencias.
			for i in range(n_tokens, len(sequence)):
				# seleccionamos la subsecuencia.
				seq = sequence[i - n_tokens:i+1]
				# y la guardamos en nuestro listado.
				sequences.append(seq)

sequences = np.array(sequences)
print('Secuencias totales:', len(sequences))

Secuencias totales: 34852


In [19]:
from sklearn.model_selection import train_test_split

# Seleccionamos los n-1 primeros tokens como input, y el último como output.
X, Y = sequences[:,:-1], sequences[:,-1:]
# Convertirmos el output a One-hot Encoding.
Y = tokenizer.sequences_to_matrix(Y)
# Generamos una partición de entrenamiento y test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [20]:
# Método para decodificar las frases tokenizadas.
def decode_sentence(sentence):
  return " ".join([tokenizer.index_word.get(w) for w in sentence])

# Seleccionamos unos cuantos inputs/ouputs para visualizarlos.
for i in np.random.choice(len(X_test), 5):
  print('Input:', decode_sequence(X_test[i]), '\nOutput:', tokenizer.index_word.get(np.argmax(Y_test[i])), '\n ---------')

Input: largos y días las tiendas y comiendo bajo de brillantes 
Output: colores 
 ---------
Input: ahora mismo por la tía petunia que era huesuda y 
Output: tenía 
 ---------
Input: hermione que había estado en el de hagrid dejó escapar 
Output: un 
 ---------
Input: lo han dicho en la porque fudge quería mantenerlo en 
Output: pero 
 ---------
Input: dejara manchas de tinta en las sábanas los dursley no 
Output: tendrían 
 ---------


In [31]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

model = Sequential()

embed_size = 100

# Capa de Entrada.
model.add(Input(shape=(n_tokens,), dtype="int32"))
# Capa de Embeddings.
model.add(Embedding(max_vocab, embed_size))
# Capa de LSTM.
model.add(Bidirectional(LSTM(units=128)))
model.add(Dropout(rate=0.2))
# Capa de Salida.
model.add(Dense(max_vocab, activation='softmax'))

callbacks = [EarlyStopping(patience=3, monitor='val_loss')]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10, callbacks=callbacks)

Epoch 1/10
730/730 [==============================] - 12s 17ms/step - loss: 6.5198 - acc: 0.0645 - val_loss: 6.2432 - val_acc: 0.0692
Epoch 2/10
730/730 [==============================] - 11s 16ms/step - loss: 5.9361 - acc: 0.0878 - val_loss: 6.0270 - val_acc: 0.0962
Epoch 3/10
730/730 [==============================] - 12s 16ms/step - loss: 5.5442 - acc: 0.1093 - val_loss: 5.9197 - val_acc: 0.1119
Epoch 4/10
730/730 [==============================] - 11s 16ms/step - loss: 5.1650 - acc: 0.1322 - val_loss: 5.8910 - val_acc: 0.1242
Epoch 5/10
730/730 [==============================] - 11s 15ms/step - loss: 4.7662 - acc: 0.1570 - val_loss: 5.9439 - val_acc: 0.1242
Epoch 6/10
730/730 [==============================] - 11s 16ms/step - loss: 4.3458 - acc: 0.1834 - val_loss: 6.0739 - val_acc: 0.1249
Epoch 7/10
730/730 [==============================] - 11s 16ms/step - loss: 3.9000 - acc: 0.2208 - val_loss: 6.2390 - val_acc: 0.1198


In [32]:
print('Input:',  decode_sentence(X_test[60]))
print('Output:', tokenizer.index_word.get(np.argmax(model.predict(X_test[60:61]))))

Input: nuestro breve encuentro fui yo quien te envió la saeta
Output: de


In [37]:
text = " ".join([tokenizer.index_word.get(w) for w in X_test[5]])
print(text)

words = text.split(" ")

for i in range(50):
  # Obtenemos la distribución de predicciones.
  tokns = tokenizer.texts_to_sequences([words])
  probs = model.predict(tokns)[0]
  # Generamos una muestra de dicha distribución para sacar la nueva palabra.
  yhat  = tokenizer.index_word.get(np.random.choice(len(probs), p=probs))
  # La añadimos al texto.
  text += " " + yhat
  # Sacamos las nuevas palabras de input.
  words = text.split(" ")[- n_tokens:]

print(text)

piensa que con su muerte quien tú sabes volvería al
piensa que con su muerte quien tú sabes volvería al de las peligrosas a recuperar el moverse el retrato y y black a la estación a el ató dando estaba bajara en una mano peluda y durmiendo de una la varita de hacerse y sin encontrara en una gata con los pesados y quedó ningún últimos maleta noticia y que


In [ ]:
np.random.choice(len(probs))

4762